In [1]:
from ETL2 import fetch_data_from_sqlite

In [2]:
results = fetch_data_from_sqlite()

In [12]:
results[:3]

[{'province': 'กระบี่',
  'amphur': 'คลองท่อม',
  'tambon': 'คลองท่อมเหนือ',
  'ballot_meta': {'division_number': '3', 'division_number_integer': 3},
  'ballot_units': [{'division_number': '3',
    'id': 39001,
    'is_verify': False,
    'total_volunteers': None,
    'unit_info': '',
    'unit_number': '1',
    'unit_number_integer': 1},
   {'division_number': '3',
    'id': 39002,
    'is_verify': False,
    'total_volunteers': None,
    'unit_info': '',
    'unit_number': '2',
    'unit_number_integer': 2},
   {'division_number': '3',
    'id': 39003,
    'is_verify': False,
    'total_volunteers': None,
    'unit_info': '',
    'unit_number': '3',
    'unit_number_integer': 3},
   {'division_number': '3',
    'id': 39004,
    'is_verify': False,
    'total_volunteers': None,
    'unit_info': '',
    'unit_number': '4',
    'unit_number_integer': 4},
   {'division_number': '3',
    'id': 39005,
    'is_verify': False,
    'total_volunteers': None,
    'unit_info': '',
    'unit_numb

In [6]:
import pandas as pd


In [13]:
df = pd.DataFrame(results)

In [15]:
df["Division"] = df.ballot_meta.apply(lambda x: x["division_number"])
df["Division2"] = df.ballot_meta.apply(lambda x: x["division_number_integer"])

In [20]:
df["Division"] = df["Division"].astype("int64")

In [22]:
(df["Division"] == df["Division2"]).all()

True

In [23]:
df.head()

,province,amphur,tambon,ballot_meta,ballot_units,Division,Division2
0,กระบี่,คลองท่อม,คลองท่อมเหนือ,"{'division_number': '3', 'division_number_inte...","[{'division_number': '3', 'id': 39001, 'is_ver...",3,3
1,กระบี่,คลองท่อม,คลองท่อมใต้,"{'division_number': '3', 'division_number_inte...","[{'division_number': '3', 'id': 38871, 'is_ver...",3,3
2,กระบี่,คลองท่อม,คลองพน,"{'division_number': '3', 'division_number_inte...","[{'division_number': '3', 'id': 39131, 'is_ver...",3,3
3,กระบี่,คลองท่อม,ทรายขาว,"{'division_number': '3', 'division_number_inte...","[{'division_number': '3', 'id': 39261, 'is_ver...",3,3
4,กระบี่,คลองท่อม,พรุดินนา,"{'division_number': '3', 'division_number_inte...","[{'division_number': '3', 'id': 39391, 'is_ver...",3,3


In [26]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Drop column: 'ballot_meta'
    df = df.drop(columns=['ballot_meta'])
    # Drop column: 'Division2'
    df = df.drop(columns=['Division2'])
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,province,amphur,tambon,ballot_units,Division
0,กระบี่,คลองท่อม,คลองท่อมเหนือ,"[{'division_number': '3', 'id': 39001, 'is_ver...",3
1,กระบี่,คลองท่อม,คลองท่อมใต้,"[{'division_number': '3', 'id': 38871, 'is_ver...",3
2,กระบี่,คลองท่อม,คลองพน,"[{'division_number': '3', 'id': 39131, 'is_ver...",3
3,กระบี่,คลองท่อม,ทรายขาว,"[{'division_number': '3', 'id': 39261, 'is_ver...",3
4,กระบี่,คลองท่อม,พรุดินนา,"[{'division_number': '3', 'id': 39391, 'is_ver...",3


In [27]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_clean):
    # Performed 2 aggregations grouped on columns: 'province', 'amphur', 'tambon'
    df_clean = df_clean.groupby(['province', 'amphur', 'tambon']).agg(ballot_units_last=('ballot_units', 'last'), Division_last=('Division', 'last')).reset_index()
    return df_clean

df_clean_1 = clean_data(df_clean.copy())
df_clean_1.head()

,province,amphur,tambon,ballot_units_last,Division_last
0,กระบี่,คลองท่อม,คลองท่อมเหนือ,"[{'division_number': '3', 'id': 39001, 'is_ver...",3
1,กระบี่,คลองท่อม,คลองท่อมใต้,"[{'division_number': '3', 'id': 38871, 'is_ver...",3
2,กระบี่,คลองท่อม,คลองพน,"[{'division_number': '3', 'id': 39131, 'is_ver...",3
3,กระบี่,คลองท่อม,ทรายขาว,"[{'division_number': '3', 'id': 39261, 'is_ver...",3
4,กระบี่,คลองท่อม,พรุดินนา,"[{'division_number': '3', 'id': 39391, 'is_ver...",3


In [43]:
df_clean_1.to_csv("ballots_nested.csv", index=False)

In [28]:
df_clean_1.Division_last.value_counts()

Division_last
2     1339
3     1177
1     1099
4      762
5      600
6      352
7      316
8      228
9      160
10     141
11      51
16      27
15      19
12      18
13      17
32      11
14       9
24       9
33       8
31       7
17       7
20       5
18       5
30       5
22       4
28       4
26       4
23       4
21       3
25       3
19       3
27       3
29       3
Name: count, dtype: int64

In [44]:
dfs = df_clean_1.ballot_units_last.apply(pd.DataFrame)

In [46]:
df_clean_1.ballot_units_last = dfs

In [49]:
def add_parent_column(x):
    # df["parent"] = df["province"] + df["amphur"] + df["tambon"]
    x.ballot_units_last["Province"] = x.province
    x.ballot_units_last["Amphur"] = x.amphur
    x.ballot_units_last["Tambon"] = x.tambon
    return x

In [50]:
df_clean_1.apply(add_parent_column, axis=1)

,province,amphur,tambon,ballot_units_last,Division_last
0,กระบี่,คลองท่อม,คลองท่อมเหนือ,division_number id is_verify total_vo...,3
1,กระบี่,คลองท่อม,คลองท่อมใต้,division_number id is_verify total_vo...,3
2,กระบี่,คลองท่อม,คลองพน,division_number id is_verify total_vo...,3
3,กระบี่,คลองท่อม,ทรายขาว,division_number id is_verify total_v...,3
4,กระบี่,คลองท่อม,พรุดินนา,division_number id is_verify total_v...,3
...,...,...,...,...,...
6398,แม่ฮ่องสอน,แม่สะเรียง,เสาหิน,division_number id is_verify total_v...,2
6399,แม่ฮ่องสอน,แม่สะเรียง,แม่คง,division_number id is_verify total_v...,2
6400,แม่ฮ่องสอน,แม่สะเรียง,แม่ยวม,division_number id is_verify total_...,2
6401,แม่ฮ่องสอน,แม่สะเรียง,แม่สะเรียง,division_number id is_verify total_...,2


In [51]:
ballot_df = pd.concat(df_clean_1.ballot_units_last.to_list())

In [56]:
ballot_df.sort_values(by=["unit_info"], ascending=False).head(100)

,division_number,id,is_verify,total_volunteers,unit_info,unit_number,unit_number_integer,Province,Amphur,Tambon
7,3,144698,False,1.0,—,8,8,ฉะเชิงเทรา,ท่าตะเกียบ,ท่าตะเกียบ
147,1,1011303,False,2.0,​ ศาลากลางบ้านหนองใหญ่,148,148,อุดรธานี,เมืองอุดรธานี,บ้านขาว
0,4,903761,False,1.0,​,1,1,สุรินทร์,รัตนบุรี,น้ำเขียว
1,2,441352,False,NaN,​,2,2,ปทุมธานี,เมืองปทุมธานี,บางขะแยง
0,4,717731,False,NaN,​,1,1,ลำปาง,แม่พริก,พระบาทวังตวง
...,...,...,...,...,...,...,...,...,...,...
2,1,813673,False,1.0,ใต้ถุนอาคารเรียนร.ร.วัดน้อยแสงจันทร์,3,3,สมุทรสงคราม,เมืองสมุทรสงคราม,ลาดใหญ่
28,1,790299,False,NaN,ใต้ถุนอาคารเรียน 3 โรงเรียนเทศบาล 2 (อ่อนอุทิศ...,29,29,สงขลา,เมืองสงขลา,บ่อยาง
3,1,388054,False,1.0,ใต้ถุนอาคารเรียน 2 โรงเรียนบ้านกำปงปีแซ ตำบลลุ...,4,4,นราธิวาส,ยี่งอ,ลุโบะบือซา
25,1,225981,True,2.0,ใต้ถุนอาคารหอพักในกำกับสวนดอก มช. ใต้อาคาร A,41,41,เชียงใหม่,เมืองเชียงใหม่,ช้างม่อย


In [53]:
ballot_df.to_csv("ballots_raw.csv", index=False)

In [36]:
vc = ballot_df.unit_info.value_counts()

In [40]:
vc[vc == 1].index.tolist()

['ศาลาอเนกประสงค์ ตำบลไฮหย่อง อำเภอพังโคน จังหวัดสกลนคร',
 'ศาลาวัดบ้านหว้าน',
 'ศาลาอเนกประสงค์บ้านห้วยเดื่อ',
 'ศาลาอเนกประสงค์บ้านป่าไม้',
 'มัสยิดบ้านเคละ',
 'โรงเรียนบ้านเตียง',
 'ศาลาวัดบ้านหนองสะมอน',
 'ศาลาอเนกประสงค์บ้านปง',
 'ศาลาอเนกประสงค์บ้านทาทุ่งหลวง',
 'บ้านดอนต่ำ หมู่5',
 'ศาลาประชาคมบ้านตะแบง ตำบลห้วยตึ๊กชู อำเภอภูสิงห์ จังหวัดศรีสะเกษ',
 'โรงเรียนราษฎร์อุทิศ (ปูแล)',
 'สร้างค้อ หมู่2',
 'บ้านแต้',
 'โรงเรียนบ้านสัน',
 'บ้านต้อน',
 'โรงเรียนบ้านกู้กู ใต้ถุนอาคารเรียนหลังใหม่(2) หมู่ที่3 ตำบลรัษฎา อำเภอเมืองภูเก็ต จังหวัดภูเก็ต',
 'ศาลากลางบ้านเมืองหลวง หมู่ที่ 2 ตำบลเมืองหลวง อำเภอห้วยทับทัน จังหวัดศรีสะเกษ',
 'ศาลาอเนกประสงค์บ้านศรีป้าน',
 'โรงเรียนบ้านแม่เมย',
 'โรงเรียนดอยครั่ง',
 'ศาลาอเนกประสงค์ บ้านหนองเบ็น ต.ไฮหย่อง อ.พังโคน จ.สกลนคร',
 'โรงเรียน ตชด. โรงงานยาสูบ 2',
 'สำนักสงฆ์แม่เหล็ก',
 'โรงเรียนบ้านลือเน็ง',
 'มหาวิทยาลัยราชภัฎภูเก็ต หอประชุม (1) หมู่ที่ 6 ตำบลรัษฎา อำเภอเมืองภูเก็ต จังหวัดภูเก็ต',
 'โรงเรียนนิคมสร้างตนเองภาคใต้ 9',
 'ศาลาประชาคมบ้านหนองดีป

In [42]:
pd.DataFrame(vc[vc == 1].index.tolist()).to_csv("unique_ballot_unit_info.csv", index=False)